In [11]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json 
from dotenv import load_dotenv
load_dotenv()
from urllib import parse


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from code_utils.utils import remove_too_long,type_score
from code_utils.BSO import update_bso_publications,get_bso_publications
from code_utils.pickle import load_cache


In [ ]:
df_ipcc=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json')

In [ ]:
df_ipcc['ipcc']=df_ipcc.apply(lambda row: {'wg': row['wg'].replace('wg',''), 'chapter': int(row['chap'][-2:].replace('_',''))}, axis=1)

In [ ]:
df_ipcc_group=df_ipcc.groupby(by=['doi'], dropna=True).agg({'ipcc': lambda x: list(x), 'countries': lambda x: list(x), 'freq': 'sum'})
df_ipcc_group=df_ipcc_group.reset_index()

In [ ]:
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: x[0])
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [ ]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization.json')

In [ ]:
df_ipbes['ipbes']=df_ipbes.apply(lambda row: {'chapter': [x.replace('chap_','') for x in row['chap']]}, axis=1)

In [ ]:
df_ipbes_group=df_ipbes.groupby(by=['doi'], dropna=True).agg({'ipbes': lambda x: list(x), 'countries': lambda x: list(x), 'freq': 'sum'})
df_ipbes_group=df_ipbes_group.reset_index()

In [ ]:
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: x[0])
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [ ]:
df_ipbes_group

In [ ]:
df_ipcc_group

In [ ]:
df_ipcc_ipbes=pd.merge(df_ipcc_group,df_ipbes_group, on='doi', how='outer')
df_ipcc_ipbes

In [ ]:
update_bso_publications()

In [ ]:
bso=pd.DataFrame(get_bso_publications())

In [ ]:
df_bso=bso.transpose().reset_index()

In [ ]:
df_bso.columns=['doi','rors', 'bso_local_affiliations']

In [ ]:
df_bso

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes,df_bso,on='doi',how='left')

In [ ]:
def get_countries(row):
    if (isinstance(row.countries_x,list)==False)&(isinstance(row.countries_y,list)==False):
        return None
    elif (isinstance(row.countries_x,list))&(isinstance(row.countries_y,list)==False):
        return row.countries_x
    elif (isinstance(row.countries_x,list)==False)&(isinstance(row.countries_y,list)):
        return row.countries_y
    else:
        return row.countries_x

In [ ]:
df_ipcc_ipbes_bso['countries']=df_ipcc_ipbes_bso.apply(lambda row: get_countries(row),axis=1)

In [ ]:
df_ipcc_ipbes_bso=df_ipcc_ipbes_bso[['doi', 'ipcc', 'ipbes', 'countries', 'rors', 'bso_local_affiliations']]

In [ ]:
cached_openalex_data = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data.pkl')
cached_openalex_data_ipbes = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_ipbes.pkl')
cached_openalex_data.update(cached_openalex_data_ipbes)
df_cache=pd.DataFrame({'doi':list(cached_openalex_data.keys()),'OpenAlex_data':list(cached_openalex_data.values())})

In [ ]:
df_cache

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_cache, on='doi', how='inner')

In [ ]:
dict_doi_bso=[]

In [ ]:
for i,row in df_ipcc_ipbes_bso.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    dict_doi_bso.append(dict_row2)

In [ ]:
len(dict_doi_bso)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)

In [ ]:
with open(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes.jsonl', 'w') as f:
      f.write(json_lines)

elastic

In [12]:
dd = pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes.jsonl', lines=True)

In [22]:
dd2=dd.dropna(subset=['ipcc']).drop_duplicates(subset=['doi'])

In [25]:
len(dd2.loc[(dd2['ipcc'].apply(lambda wg_chap: any('1' in x.get('wg', '') for x in wg_chap)))])

12819

In [3]:
current_dict = dd.to_dict(orient='records')

In [4]:
list_bso_ipcc_ipbes=[]
for i in range(len(dd)):
    if len(current_dict[i]['OpenAlex_data'])==1:
        openalex_info = current_dict[i]['OpenAlex_data'][0]
    else:
        openalex_info = current_dict[i]['OpenAlex_data'][1]
    del current_dict[i]['OpenAlex_data']
    for f in openalex_info:
        current_dict[i]['openalex_'+f] = openalex_info[f]  

In [5]:
df_elastic_input=pd.DataFrame(current_dict)

In [6]:
df_elastic_input.loc[:,'openalex_concepts']=df_elastic_input.loc[:,'openalex_concepts'].apply(type_score)

In [7]:
del df_elastic_input['openalex_abstract_inverted_index']

In [8]:
len(df_elastic_input)

58520

In [9]:
dict_doi_bso=[]
for i,row in df_elastic_input.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    #dict_row3=remove_too_long(dict_row2)
    dict_doi_bso.append(dict_row2)

In [10]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)
with open('bso_ipcc_ipbes_elastic.jsonl', 'w') as f:
      f.write(json_lines)

In [11]:
#df=pd.read_json('bso_ipcc_ipbes_elastic.jsonl', lines=True)

In [ ]:
es_url_without_http = os.getenv("ES_URL").replace('https://','').replace('http://','')
es_host = f'https://{os.getenv("ES_LOGIN_TEDS_BACK")}:{parse.quote(os.getenv("ES_PASSWORD_TEDS_BACK"))}@{es_url_without_http}'
index_name='teds-bibliography'